In [2]:
import numpy as np
import pandas as pd
from parser_random_exp import parse_results
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data, params = parse_results()

         exp_id _try                name step     value
0  01969828-3fe    1  vq_code_usage_frac  149  0.775890
1  01969828-3fe    1               f1/f1  149  0.282598
2  01969828-3fe    1        f1/precision  149  0.370551
3  01969828-3fe    1           f1/recall  149  0.231720
4  01969828-3fe    1  vq_code_usage_frac  149  0.775890


In [73]:
met = 'many_es_to_one_gt_accuracy/gan_tokens'
res = data[data['name'] == met].round(2)
best_for_run = res.groupby(['exp_id', '_try', 'name'])['exp_id', '_try', 'name', 'value'].max()

In [6]:
# res.groupby(['exp_id', '_try']).describe().sort_values(('value', 'max'), ascending=False)

value                                                    \
                  count   mean           std   min     25%    50%     75%   
exp_id       _try                                                           
5614a836-81b 0     10.0  0.301  3.071373e-02  0.22  0.3000  0.310  0.3175   
dfd1849b-3db 0     10.0  0.294  3.062316e-02  0.22  0.2850  0.305  0.3100   
5614a836-81b 1     10.0  0.269  3.541814e-02  0.20  0.2600  0.270  0.2875   
73394c81-d47 0     10.0  0.295  1.900292e-02  0.25  0.2925  0.300  0.3000   
47637c3d-39f 0     10.0  0.268  3.614784e-02  0.19  0.2550  0.275  0.2900   
e5688550-beb 0     10.0  0.275  2.013841e-02  0.23  0.2700  0.275  0.2800   
673d2610-1b1 0     10.0  0.249  5.300943e-02  0.16  0.2075  0.265  0.2850   
73394c81-d47 1     10.0  0.290  1.490712e-02  0.26  0.2825  0.290  0.2975   
b448c850-5ec 0     10.0  0.277  4.667857e-02  0.18  0.2900  0.295  0.3000   
             1     10.0  0.271  4.724640e-02  0.16  0.2650  0.290  0.3000   
dfd1849b-3db 1     10.0  0.281  2.469818e-02  0.24  0.2800  0.280  0.2975   
b4fdec6f-c51 1     10.0  0.247  3.917199e-02  0.16  0.2325  0.240  0.2775   
3dc83b32-8c6 1     10.0  0.270  2.309401e-02  0.22  0.2700  0.280  0.2800   
b04e859e-136 1     10.0  0.278  1.316561e-02  0.25  0.2725  0.280  0.2800   
323643a6-d05 1     10.0  0.270  1.885618e-02  0.23  0.2700  0.270  0.2800   
9b898bb4-636 1     10.0  0.277  2.710064e-02  0.23  0.2675  0.290  0.2900   
580dcf23-4d1 1     10.0  0.271  3.281260e-02  0.19  0.2700  0.275  0.2950   
673d2610-1b1 1     10.0  0.253  4.691600e-02  0.16  0.2450  0.270  0.2800   
c7e7e565-f7b 0     10.0  0.274  1.646545e-02  0.25  0.2600  0.275  0.2875   
             1     10.0  0.277  1.766981e-02  0.25  0.2600  0.285  0.2900   
10456762-911 1     10.0  0.270  1.333333e-02  0.26  0.2600  0.265  0.2775   
dba698bc-a13 1     10.0  0.267  3.560587e-02  0.18  0.2600  0.270  0.2900   
8b961707-b09 1     10.0  0.241  4.817791e-02  0.17  0.2025  0.240  0.2875   
             0     10.0  0.242  5.349974e-02  0.16  0.1925  0.265  0.2850   
0b4ce0a5-8cf 1     10.0  0.270  2.000000e-02  0.23  0.2600  0.280  0.2800   
0a322081-058 1     10.0  0.273  2.057507e-02  0.24  0.2700  0.275  0.2800   
7659fb62-c44 1     10.0  0.271  1.663330e-02  0.25  0.2600  0.270  0.2700   
09821cbb-08f 0     10.0  0.250  3.711843e-02  0.17  0.2275  0.265  0.2700   
e5688550-beb 1     10.0  0.282  1.873796e-02  0.24  0.2725  0.285  0.2975   
94230ce9-488 1     10.0  0.254  3.777124e-02  0.18  0.2300  0.265  0.2800   
...                 ...    ...           ...   ...     ...    ...     ...   
e23fcb2b-8ed 1     10.0  0.218  2.699794e-02  0.17  0.2025  0.220  0.2375   
09821cbb-08f 1     10.0  0.233  2.311805e-02  0.18  0.2225  0.240  0.2400   
6eb373fc-af0 0     10.0  0.216  2.674987e-02  0.18  0.1925  0.215  0.2300   
df2f8f57-d60 1     10.0  0.197  3.591657e-02  0.16  0.1625  0.190  0.2275   
dba698bc-a13 0     10.0  0.244  9.660918e-03  0.23  0.2400  0.245  0.2500   
c91c310d-5d9 0     10.0  0.245  2.223611e-02  0.20  0.2425  0.255  0.2600   
df2f8f57-d60 0     10.0  0.224  2.011080e-02  0.20  0.2100  0.225  0.2300   
fe62d3c9-cdf 1     10.0  0.228  3.119829e-02  0.17  0.2125  0.230  0.2575   
a86a4ea8-fe0 1     10.0  0.229  3.178050e-02  0.17  0.2150  0.245  0.2500   
c91c310d-5d9 1     10.0  0.224  2.716207e-02  0.17  0.2225  0.235  0.2400   
62daa09b-199 0     10.0  0.175  2.549510e-02  0.16  0.1600  0.160  0.1800   
5ef3f452-162 0     10.0  0.203  1.946507e-02  0.18  0.1925  0.200  0.2075   
5434d89e-499 1     10.0  0.218  3.011091e-02  0.16  0.1975  0.235  0.2400   
9f1775b6-4b2 0     10.0  0.197  3.198958e-02  0.16  0.1800  0.185  0.2300   
725ca2be-d15 1     10.0  0.188  2.573368e-02  0.16  0.1625  0.190  0.2000   
5ef3f452-162 1     10.0  0.195  2.273030e-02  0.16  0.1825  0.195  0.2000   
6eb373fc-af0 1     10.0  0.183  2.359378e-02  0.16  0.1625  0.180  0.1950   
0be74247-d85 0     10.0  0.192  2.201010e-02  0.16  0.1725  0.190  0.2100   
a1cdccfd-7c5 0

In [41]:
joined = pd.merge(params, res,left_index=True, right_on='exp_id',  how='outer')
joined.head()

,hidden_1_size,hidden_2_size,maxpool_reduction,gradient_penalty_ratio,lr,exp_id,_try,name,step,value
27,1024,2048,16,0.5,0.0016,01969828-3fe,1,many_es_to_one_gt_accuracy/gan_tokens,149,0.24
60,1024,2048,16,0.5,0.0016,01969828-3fe,1,many_es_to_one_gt_accuracy/gan_tokens,298,0.24
93,1024,2048,16,0.5,0.0016,01969828-3fe,1,many_es_to_one_gt_accuracy/gan_tokens,447,0.26
126,1024,2048,16,0.5,0.0016,01969828-3fe,1,many_es_to_one_gt_accuracy/gan_tokens,596,0.27
159,1024,2048,16,0.5,0.0016,01969828-3fe,1,many_es_to_one_gt_accuracy/gan_tokens,745,0.26


In [43]:
joined.groupby('maxpool_reduction')['value'].describe()

,count,mean,std,min,25%,50%,75%,max
maxpool_reduction,,,,,,,,
2,300.0,0.237467,0.047895,0.16,0.19,0.26,0.2700,0.33
4,120.0,0.268250,0.024277,0.16,0.26,0.27,0.2800,0.32
8,217.0,0.227834,0.042941,0.16,0.19,0.24,0.2600,0.30
16,275.0,0.240255,0.044050,0.16,0.21,0.26,0.2700,0.31
32,200.0,0.259900,0.036340,0.16,0.25,0.27,0.2825,0.32


In [51]:
joined.groupby(['hidden_2_size', 'maxpool_reduction'])['value'].describe().sort_values('max')

count      mean       std   min     25%  \
hidden_2_size maxpool_reduction                                            
1024          8                   20.0  0.183000  0.027164  0.16  0.1600   
2048          32                  20.0  0.234500  0.026453  0.17  0.2275   
128           4                   20.0  0.247000  0.030625  0.16  0.2500   
2048          16                  40.0  0.225750  0.042299  0.16  0.1800   
128           16                  60.0  0.262833  0.015741  0.22  0.2575   
512           2                   40.0  0.208500  0.052406  0.16  0.1600   
2048          8                   60.0  0.218667  0.047495  0.16  0.1600   
1024          16                  75.0  0.240667  0.043069  0.16  0.2100   
              4                   20.0  0.274500  0.012344  0.25  0.2700   
              2                   60.0  0.238833  0.036643  0.16  0.2000   
128           2                  100.0  0.241500  0.045558  0.16  0.2175   
256           16                  40.0  0.231750  0.040818  0.16  0.1975   
              8                   97.0  0.237423  0.036976  0.16  0.2200   
              4                   60.0  0.266833  0.017123  0.22  0.2600   
512           8                   40.0  0.240750  0.038921  0.16  0.2175   
              16                  60.0  0.232500  0.058153  0.16  0.1600   
256           32                  80.0  0.261375  0.031892  0.16  0.2500   
1024          32                  60.0  0.257000  0.043891  0.16  0.2375   
128           32                  40.0  0.274000  0.029509  0.18  0.2500   
512           4                   20.0  0.287500  0.027886  0.22  0.2800   
2048          2                  100.0  0.244200  0.050676  0.16  0.2075   

                                   50%     75%   max  
hidden_2_size maxpool_reduction                       
1024          8                  0.180  0.1825  0.24  
2048          32                 0.240  0.2525  0.26  
128           4                  0.255  0.2600  0.28  
2048          16                 0.240  0.2625  0.28  
128           16                 0.270  0.2700  0.29  
512           2                  0.170  0.2600  0.29  
2048          8                  0.235  0.2600  0.30  
1024          16                 0.260  0.2700  0.30  
              4                  0.275  0.2800  0.30  
              2                  0.250  0.2700  0.30  
128           2                  0.260  0.2700  0.30  
256           16                 0.230  0.2625  0.30  
              8                  0.240  0.2700  0.30  
              4                  0.265  0.2800  0.30  
512           8                  0.255  0.2700  0.30  
              16                 0.260  0.2800  0.31  
256           32                 0.270  0.2800  0.31  
1024          32                 0.270  0.2900  0.31  
128           32                 0.280  0.3000  0.32  
512           4                  0.295  0.3100  0.32  
2048          2                  0.260  0.2800  0.33

(1112, 10)

In [94]:
(
        pd.merge(params, best_for_run,left_index=True, right_on='exp_id',  how='outer')
            .groupby(['hidden_1_size', 'hidden_2_size', 'maxpool_reduction'])['value']
            .describe()
            .sort_values('mean', ascending=False)
)


count      mean       std  \
hidden_1_size hidden_2_size maxpool_reduction                              
2048          512           4                    2.0  0.315000  0.007071   
128           512           16                   2.0  0.310000  0.000000   
256           1024          32                   2.0  0.305000  0.007071   
2048          1024          32                   4.0  0.302500  0.005000   
128           256           32                   2.0  0.300000  0.014142   
256           256           4                    2.0  0.300000  0.000000   
512           128           32                   4.0  0.297500  0.026300   
                            2                    2.0  0.295000  0.007071   
1024          1024          4                    2.0  0.295000  0.007071   
512           512           8                    2.0  0.295000  0.007071   
              256           32                   2.0  0.295000  0.007071   
128           256           4                    2.0  0.290000  0.000000   
512           1024          16                   2.0  0.290000  0.000000   
256           256           16                   2.0  0.285000  0.021213   
                            32                   2.0  0.285000  0.007071   
1024          256           8                    2.0  0.280000  0.000000   
2048          2048          8                    2.0  0.280000  0.028284   
1024          128           16                   2.0  0.280000  0.014142   
512           512           2                    2.0  0.280000  0.014142   
2048          128           16                   4.0  0.277500  0.005000   
512           256           4                    2.0  0.275000  0.007071   
1024          256           32                   2.0  0.275000  0.007071   
              1024          16                   4.0  0.275000  0.017321   
              2048          16                   2.0  0.275000  0.007071   
2048          128           4                    2.0  0.275000  0.007071   
128           256           8                    6.0  0.270000  0.023664   
              2048          2                    2.0  0.270000  0.014142   
2048          1024          2                    6.0  0.266667  0.027325   
              2048          2                    8.0  0.266250  0.067599   
1024          128           2                    2.0  0.265000  0.007071   
256           512           8                    2.0  0.260000  0.000000   
              256           8                    2.0  0.260000  0.028284   
2048          2048          32                   2.0  0.255000  0.007071   
              128           2                    6.0  0.245000  0.059582   
1024          256           16                   2.0  0.245000  0.021213   
2048          2048          16                   2.0  0.245000  0.035355   
512           512           16                   4.0  0.225000  0.069522   
256           2048          8                    4.0  0.217500  0.066521   
2048          1024          8                    2.0  0.210000  0.042426   
256           1024          16                   2.0  0.200000  0.056569   
1024          512           2                    2.0  0.160000  0.000000   

                                                min     25%    50%     75%  \
hidden_1_size hidden_2_size maxpool_reduction                                
2048          512           4                  0.31  0.3125  0.315  0.3175   
128           512           16                 0.31  0.3100  0.310  0.3100   
256           1024          32                 0.30  0.3025  0.305  0.3075   
2048          1024          32                 0.30  0.3000  0.300  0.3025   
128           256           32                 0.29  0.2950  0.300  0.3050   
256           256           4                  0.30  0.3000  0.300  0.3000   
512           128           32                 0.26  0.2900  0.305  0.3125   
                            2                  0.29  0.2925  0.295  0.2975  